# Errores de codificación a nivel de mesa electoral

Prueba de concepto.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
from common_functions import get_parties, get_election_results, process_election

Estos archivos provienen de http://www.infoelectoral.mir.es/infoelectoral/min/areaDescarga.html. Para acceder a ellos hay que ir al final de la página, hasta la sección *Extracción de datos*, y luego elegir los que nos interesen. Los que empiezan por *02* son elecciones municipales, *04* generales, etc. Dentro de cada .zip hay un archivo DOCX / RTF con instrucciones sobre los distintos códigos.

In [3]:
datafiles = sorted(glob.glob("/home/chema/Dropbox/data/elecciones/*.zip"))
for x in enumerate(datafiles):
    print(x)

(0, '/home/chema/Dropbox/data/elecciones/02201512_MESA.zip')
(1, '/home/chema/Dropbox/data/elecciones/02201606_MESA.zip')
(2, '/home/chema/Dropbox/data/elecciones/04201105_MESA.zip')
(3, '/home/chema/Dropbox/data/elecciones/04201505_MESA.zip')


De acuerdo a [este archivo](http://www.ine.es/daco/daco42/codmun/codmun11/11codmunmapa.htm), Madrid es la provincia 28, municipio 079. De momento no nos interesa otra cosa. La función `process_election` los usa por defecto.

In [4]:
# Select index according to list above, then run analysis
file_idx = 2
eldata = process_election(datafiles[file_idx])
print(eldata.shape)

(83675, 8)


In [5]:
# Need to get the percentage of vote for a given "table" (ballot box) + section
eldata_total = eldata.groupby(['section_code', 'table_code', 'dist_code'])\
    .agg({'votes': 'sum'})\
    .rename(columns = {'votes': 'total_votes'})\
    .reset_index()
eldata = pd.merge(eldata, eldata_total)
eldata['pct_votes'] = eldata['votes'] / eldata['total_votes']

En el análisis [que hice hace un tiempo](https://rinzewind.org/blog-es/2015/codigo-preliminar-para-analisis-de-elecciones-municipales-por-mesas.html) había mesas con un voto desproporcionado a *LA FALANGE*. Podemos probar a buscar sólo esas, a ver qué pasa.

In [6]:
# Now find the boxes that had the most vote to Falange
eldata.loc[eldata['party_name'] == "LA FALANGE", :].sort_values('pct_votes', ascending = False).head(5)

,dist_code,party_code,party_name,prov_code,section_code,table_code,town_code,votes,total_votes,pct_votes
27283,08,122620,LA FALANGE,28,012,U,079,349,702,0.497151
43033,11,122620,LA FALANGE,28,007,B,079,184,396,0.464646
57333,13,122620,LA FALANGE,28,203,A,079,84,404,0.207921
53683,13,122620,LA FALANGE,28,061,U,079,5,301,0.016611
1733,01,122620,LA FALANGE,28,062,U,079,8,487,0.016427


Podemos explorar cualquier mesa cambiando `dist_code`, `section_code` y `table_code`. Aquí el ejemplo es la que sale con más porcentaje en la lista de arriba.

In [7]:
dist_code = '08'
section_code = '012'
table_code = 'U'
summary = eldata.loc[(eldata['section_code'] == section_code) \
                      & (eldata['dist_code'] == dist_code) \
                      & (eldata['table_code'] == table_code), :]\
    .sort_values('pct_votes', ascending = False)
display(summary)

,dist_code,party_code,party_name,prov_code,section_code,table_code,town_code,votes,total_votes,pct_votes
27283,08,122620,LA FALANGE,28,012,U,079,349,702,0.497151
27294,08,123438,POR UN MUNDO MAS JUSTO,28,012,U,079,184,702,0.262108
27284,08,122876,PARTIDO ANTITAURINO CONTRA EL MALTRATO ANIMAL,28,012,U,079,85,702,0.121083
27297,08,123771,UNION POR LEGANES,28,012,U,079,56,702,0.079772
27282,08,122515,IZQUIERDA UNIDA-LOS VERDES,28,012,U,079,8,702,0.011396
27277,08,121193,CIUDADANOS-PARTIDO DE LA CIUDADANIA,28,012,U,079,4,702,0.005698
27289,08,122987,PARTIDO HUMANISTA,28,012,U,079,3,702,0.004274
27281,08,121716,FORO CENTRO Y DEMOCRACIA,28,012,U,079,2,702,0.002849
27293,08,123186,PARTIDO SOCIALISTA OBRERO ESPAÑOL,28,012,U,079,2,702,0.002849
27286,08,122895,PARTIDO COMUNISTA DE LOS PUEBLOS DE ESPAÑA,28,012,U,079,2,702,0.002849


Para comprobar si la lectura de datos está bien hecha, podemos mirar el total por partido. Estas cifras tienen que coincidir con las que se pueden obtener [aquí](http://www.infoelectoral.mir.es/infoelectoral/min/busquedaAvanzadaAction.html) o en el BOE correspondiente.

In [8]:
eldata.groupby('party_name')\
    .agg({'votes': 'sum'})\
    .reset_index()\
    .sort_values('votes', ascending = False)

,party_name,votes
18,PARTIDO POPULAR,756952
19,PARTIDO SOCIALISTA OBRERO ESPAÑOL,364600
8,IZQUIERDA UNIDA-LOS VERDES,163706
24,UNION PROGRESO Y DEMOCRACIA,119601
3,ECOLO VERDES,13425
1,CIUDADANOS EN BLANCO,10795
10,PARTIDO ANTITAURINO CONTRA EL MALTRATO ANIMAL,7071
20,POR UN MUNDO MAS JUSTO,6456
0,ALTERNATIVA ESPAÑOLA,4764
17,PARTIDO PIRATA,4631


Una forma fácil de encontrar mesas mal codificadas es localizar aquellas que tienen algún partido con más de 20 veces su porcentaje medio. Es un método basto pero salen cosas interesantes.

In [9]:
eldata_stats = eldata.groupby('party_name')\
    .agg({'pct_votes': 'mean'})\
    .rename(columns = {'pct_votes': 'mean'})\
    .reset_index()
eldata_stats['threshold'] = 20 * eldata_stats['mean']
eldata_with_stats = pd.merge(eldata, eldata_stats)
bad_data = eldata_with_stats.loc[(eldata_with_stats['pct_votes'] > eldata_with_stats['threshold']) & (eldata_with_stats['votes'] > 20), :]\
    .groupby(['dist_code', 'section_code', 'table_code'])\
    .agg({'pct_votes': 'count'})\
    .rename(columns = {'pct_votes': 'bad_counts'})\
    .reset_index()\
    .sort_values('bad_counts', ascending = False)
display(bad_data)

,dist_code,section_code,table_code,bad_counts
4,08,012,U,4
0,01,062,U,3
9,13,203,A,3
6,11,007,B,2
1,03,078,B,1
2,05,091,U,1
3,05,093,U,1
5,09,018,B,1
7,12,101,A,1
8,13,060,U,1
